# PyTorch 有何业务？

在本课之前的学习当中，我们已经编写了大量代码来完成一套神经网络功能。Dropout、Batch Norm 和 2D 卷积是计算机视觉深度学习的一些主力。我们还努力使代码高效且矢量化。

不过，对于本次作业的后一部分，我们将放弃您漂亮的代码库，而是迁移到两个流行的深度学习框架之一：在本例中，是 PyTorch（或 TensorFlow，如果您切换到该Jupyter Notebook）。 

### 什么是 PyTorch?

PyTorch 是一个用于在 Tensor 对象上执行动态计算图的系统，其行为与 Numpy Ndarray 类似。它配备了强大的自动微分引擎，无需手动反向传播。

### 为什么用PyTorch?

* 我们的代码现在将在 GPU 上运行！训练速度更快。使用 PyTorch 或 TensorFlow 等框架时，您可以利用 GPU 的强大功能来实现您自己的自定义神经网络架构，而无需直接编写 CUDA 代码（这超出了本课程的范围）。
* 我们希望您准备好在您的项目中使用这些框架之一，这样您就可以比手动编写要使用的每个功能更有效地进行实验。
* 我们希望你站在巨人的肩膀上！ TensorFlow 和 PyTorch 都是优秀的框架，可以让您的生活变得更加轻松，现在您了解了它们的本质，就可以自由使用它们了🙂
* 我们希望您能够接触到在学术界或工业界可能遇到的深度学习代码。

### PyTorch 版本

本笔记本假设您使用 **PyTorch 0.4 版本**.在此版本之前，张量必须包装在 Variable 对象中才能在 Autograd 中使用；然而，变量现在已被弃用。此外，0.4 还将 Tensor 的数据类型与其设备分离，并使用 Numpy 风格的工厂来构造 Tensor，而不是直接调用 Tensor 构造函数。

## 我们将如何学习 PyTorch?

Justin Johnson 为PyTorch做了杰出贡献[教程](https://github.com/jcjohnson/pytorch-examples)。

你还可以在这里找到详细的[API文档](http://pytorch.org/docs/stable/index.html) 。 如果您有 API 文档未解决的其他问题,[PyTorch forum](https://discuss.pytorch.org/) 是一个比 StackOverflow 更好的提问地点。


# 目录

本作业有 5 个部分。您将在不同的抽象级别上学习 PyTorch，这将帮助您更好地理解它并为最终项目做好准备。

1. 准备工作: 我们将使用 CIFAR-10 数据集.
2. Barebons PyTorch: 我们将直接使用最低级别的 PyTorch 张量。 
3. PyTorch Module API: 我们将使用 `nn.Module` 来定义任意神经网络架构。
4. PyTorch Sequential API: 我们将使用 `nn.Sequential` 非常方便地定义线性前馈网络。
5. CIFAR-10 开放式挑战: 请实现您自己的网络，以在 CIFAR-10 上获得尽可能高的准确性。您可以尝试任何层、优化器、超参数或其他高级功能。

这是一个比较表:

| API           | 灵活性 | 便捷度 |
|---------------|-------------|-------------|
| Barebone      | 高        | 低         |
| `nn.Module`     | 高        | 中      |
| `nn.Sequential` | 低         | 高        |

# Part I. 准备工作
首先，我们加载 CIFAR-10 数据集。第一次执行此操作可能需要几分钟，但此后文件应保留在缓存中。

在作业的前面部分中，我们必须编写自己的代码来下载 CIFAR-10 数据集、对其进行预处理并以小批量方式迭代它； PyTorch 提供了方便的工具来为我们自动化这个过程。

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [3]:
NUM_TRAIN = 49000

# torchvision.transforms 包提供了用于预处理数据的工具以及用于执行数据增强；在这里我们设置了一个转换
# 通过减去平均 RGB 值并除以每个RGB值的标准差；我们已经对平均值和标准差进行了硬编码。
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# 我们为每个分割（train / val / test）设置一个Dataset对象；数据集一次加载一个训练示例
# 因此，我们将每个数据集包装在 DataLoader 中，该数据加载器迭代数据集并形成小批量。
# 我们通过将 Sampler 对象传递给 CIFAR-10 训练集分为训练集和验证集。
# DataLoader 告诉它应该如何从底层数据集中采样。
cifar10_train = dset.CIFAR10('./ds204-lab/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./ds204-lab/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./ds204-lab/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 170498071/170498071 [12:36<00:00, 225376.10it/s]


Extracting ./ds204-lab/datasets\cifar-10-python.tar.gz to ./ds204-lab/datasets
Files already downloaded and verified
Files already downloaded and verified


您可以选择 **通过将下面USE_GPU的标志设置为 True 来使用 GPU**. 此任务无需使用 GPU。请注意，如果您的计算机未启用CUDA , `torch.cuda.is_available()` 将返回 False，并且此笔记本将回退到 CPU 模式。

全局变量 `dtype` 和 `device`将控制整个分配过程中的数据类型。 

In [4]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


# Part II. Barebones PyTorch

PyTorch 附带了高级 API，可以帮助我们方便地定义模型架构，我们将在本教程的第二部分中介绍这一点。在本节中，我们将从准系统 PyTorch 元素开始，以更好地了解 Autograd 引擎。完成本次练习后，您将更加了解高级模型 API。

我们将从一个简单的全连接 ReLU 网络开始，该网络具有两个隐藏层并且对 CIFAR 分类没有偏差。
此实现使用 PyTorch 张量上的操作来计算前向传递，并使用 PyTorch Autograd 来计算梯度。理解每一行很重要，因为在示例之后您将编写一个更难的版本。

当我们创建 PyTorch 张量时 `requires_grad=True`,那么涉及该张量的运算将不仅仅是计算值；他们还将在后台建立一个计算图，使我们能够轻松地通过该图进行反向传播，以计算一些张量相对于下游损失的梯度。 具体来说，如果 x 是一个张量 `x.requires_grad == True` 那么在反向传播之后，`x.grad` 将是另一个张量，它保存着 x 相对于最后的标量损失的梯度。

### PyTorch 张量(Tensor): 展平函数
PyTorch 张量在概念上类似于 numpy 数组：它是一个 n 维数字网格，并且与 numpy 一样，PyTorch 提供了许多函数来有效地对张量进行操作。作为一个简单的例子，我们提供了一个 `(展平)flatten` 函数，它可以重塑图像数据以用于全连接的神经网络。

回想一下，图像数据通常存储在形状为 N x C x H x W 的张量中，其中：

* N 是数据点的数量
* C是通道数
* H 是中间特征图的高度（以像素为单位）
* W 是中间特征图的宽度（以像素为单位）

当我们进行 2D 卷积之类的操作时，这是表示数据的正确方法，这需要对中间特征彼此相对的位置进行空间理解。 然而，当我们使用全连接的仿射层来处理图像时，我们希望每个数据点都由单个向量表示 - 隔离数据的不同通道、行和列不再有用。 因此，我们使用“展平”操作将每个表示的“C x H x W”值折叠为单个长向量。 下面的 flatten 函数首先从给定的一批数据中读取 N、C、H 和 W 值，然后返回该数据的“视图”。 “View”类似于 numpy 的“reshape”方法：它将 x 的维度重塑为 N x ??，其中 ??允许是任何东西（在本例中，它将是 C x H x W，但我们不需要明确指定）。

In [5]:
def flatten(x):
    N = x.shape[0] # 读取 N, C, H, W
    return x.view(N, -1)  # "展平"  C * H * W 的值，转化为每个图像的单个向量

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


### Barebones PyTorch: 双层网络

这里我们定义了一个函数 `two_layer_fc`，它对一批图像数据执行两层全连接 ReLU 网络的前向传递。定义前向传递后，我们检查它是否不会崩溃，并且通过在网络中运行零来生成正确形状的输出。

您不必在这里编写任何代码，但阅读并理解实现非常重要。

In [6]:
import torch.nn.functional as F  # 有用的无状态函数

def two_layer_fc(x, params):
    """
    全连接的神经网络；
    其框架是:
    NN 为  全连接-> ReLU -> 全连接层。
    注意这个函数只定义了前向传播；
    PyTorch 将为我们处理向后传递。
    
    网络的输入将是一小批数据， 其形状
    (N, d1, ..., dM) 其中 d1 * ... * dM = D。 T隐藏层将有 H 个单元，
    输出层将产生 C 类的分数。
    
    输入:
    - x: 形状为 (N, d1, ..., dM) 的 PyTorch 张量，给出的小批量输入数据。
    - params:PyTorch 张量列表 [w1, w2] 为网络提供权重；w1 具有形状 (D, H)，w2 具有形状 (H, C)。
    
    返回:
    - scores: 形状为 (N, C) 的 PyTorch 张量，给出输入数据x的分类分数。
    """
    # 首先我们展平图像
    x = flatten(x)  # shape: [batch_size, C x H x W]
    
    w1, w2 = params
    
    # 前向传递: 使用张量运算计算预测 y. w1和w2的 requires_grad=True，
    # 涉及这些Tensors的操作会导致PyTorch 构建计算图，允许自动计算梯度
    # 因此我们不再手动实现向后传递 ，我们不需要保留对中间值的引用。
    # 你也可以使用`.clamp(min=0)`, 相当于F.relu()
    x = F.relu(x.mm(w1))
    x = x.mm(w2)
    return x
    

def two_layer_fc_test():
    hidden_layer_size = 42
    x = torch.zeros((64, 50), dtype=dtype)  # 小批量大小 64,特征维度 50
    w1 = torch.zeros((50, hidden_layer_size), dtype=dtype)
    w2 = torch.zeros((hidden_layer_size, 10), dtype=dtype)
    scores = two_layer_fc(x, [w1, w2])
    print(scores.size())  # 你应该看到输出为 [64, 10]

two_layer_fc_test()

torch.Size([64, 10])


### Barebones PyTorch: 三层卷积网络

在这里，您将完成函数 `three_layer_convnet`的实现, 该函数将执行三层卷积网络的向前传递。 像上面一样，我们可以通过网络传递零来立即测试我们的实现。网络应具有以下架构：
1. 带有`channel_1`滤波器的卷积层（带有偏差），每个滤波器的形状为`KW1 x KH1`，以及两个零填充
2. 非线性ReLU
3. 带有`channel_2`滤波器的卷积层（带有偏差），每个滤波器的形状为`KW2 x KH2`，以及两个零填充
4. 非线性ReLU
5. 带偏差的全连接层，为 C 类生成分数。

**提示**: 对于卷积: http://pytorch.org/docs/stable/nn.html#torch.nn.functional.conv2d; 注意卷积滤波器的形状！

In [8]:
def three_layer_convnet(x, params):
    """
    使用上面定义的架构执行三层卷积网络的前向传递。

    输入:
    - x: 提供小批量图像形状 (N, 3, H, W) 的 PyTorch 张量，
    - params: 给出网络权重和偏差的 PyTorch 张量列表；
    应包含以下内容：
      - conv_w1：形状 (channel_1, 3, KH1, KW1) 的 PyTorch 张量，给出第一个卷积层的权重
      - conv_b1: 形状 (channel_1,) 的 PyTorch 张量给出第一个卷积层的偏差
      - conv_w2: 形状 (channel_2, channel_1, KH2, KW2) 的 PyTorch 张量，给出第二个卷积层的权重
      - conv_b2: 形状 (channel_2,) 的 PyTorch 张量给出第二个卷积层的偏差
      - fc_w: PyTorch Tensor 为全连接层赋予权重。你能猜出它应该是什么形状吗？
      - fc_b: PyTorch Tensor 为全连接层提供偏差。你能猜出它应该是什么形状吗？
    
    返回:
    - scores: 形状 (N, C) 的 PyTorch 张量给出 x 的分类分数
    """
    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params
    scores = None
    ################################################################################
    # TODO: 实现三层 ConvNet 的前向传递。                                           #
    ################################################################################
    #       *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****          #
    ################################################################################
    
  
    conv1 = F.conv2d(x, weight=conv_w1, bias=conv_b1, padding=2)
    relu1 = F.relu(conv1)
    conv2 = F.conv2d(relu1, weight=conv_w2, bias=conv_b2, padding=1)
    relu2 = F.relu(conv2)
    relu2_flat = flatten(relu2)
    scores = relu2_flat.mm(fc_w) + fc_b
    
    ################################################################################
    #                                END OF YOUR CODE                              #
    ################################################################################
    return scores

定义上述 ConvNet 的前向传递后，运行以下单元来测试您的实现。

当您运行此函数时，scores的形状应为 (64, 10)。

In [9]:
def three_layer_convnet_test():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # 小批量大小 64, 图片大小 [3, 32, 32]

    conv_w1 = torch.zeros((6, 3, 5, 5), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b1 = torch.zeros((6,))  # out_channel
    conv_w2 = torch.zeros((9, 6, 3, 3), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b2 = torch.zeros((9,))  # out_channel

    # 您必须在两个转换层之后、全连接层之前计算张量的形状
    fc_w = torch.zeros((9 * 32 * 32, 10))
    fc_b = torch.zeros(10)

    scores = three_layer_convnet(x, [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b])
    print(scores.size())  # 你应该看到[64, 10]
three_layer_convnet_test()

torch.Size([64, 10])


### Barebones PyTorch: 初始化
让我们编写几个实用方法来初始化模型的权重矩阵。

- `random_weight(shape)` 使用 Kaiming 归一化方法初始化权重张量。
- `zero_weight(shape)` 用全零初始化权重张量。对于实例化偏置参数很有用。

`random_weight` 函数使用Kaiming普通初始化方法，描述如下

He et al, *Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification*, ICCV 2015, https://arxiv.org/abs/1502.01852

In [11]:
def random_weight(shape):
    """
   为权重创建随机张量；设置 requires_grad=True 意味着我们
    想要在向后传递过程中计算这些张量的梯度。
    我们使用 Kaiming 归一化: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn 是标准正态分布生成器。 
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# 创建权重的形状 [3 x 5]
# 如果您使用 GPU，您应该会看到`torch.cuda.FloatTensor`类型。
# 否则它应该是  `torch.FloatTensor`
random_weight((3, 5))

tensor([[ 0.4758, -0.7288, -0.7309, -0.6499,  0.1560],
        [ 0.1188,  0.2502, -0.4702,  0.5564, -0.2477],
        [-1.1646, -1.2569,  0.6985, -0.6833, -1.2493]], device='cuda:0',
       requires_grad=True)

### Barebones PyTorch: 检查准确性
训练模型时，我们将使用以下函数来检查模型在训练或验证集上的准确性。

检查准确性时，我们不需要计算任何梯度；因此，当我们计算分数时，我们不需要 PyTorch 为我们构建计算图。为了防止构建图表，我们将计算范围限制在`torch.no_grad()`上下文管理器下。

In [12]:
def check_accuracy_part2(loader, model_fn, params):
    """
   检查分类模型的准确性。
    
    输入:
    - loader:用于我们要检查的数据分割的 DataLoader
    - model_fn:执行模型前向传递的函数，签名为 Score = model_fn(x, params)
    - params: 给出模型参数的 PyTorch 张量列表
    
    返回:Nothing, 但打印模型的准确性
    """
    split = 'val' if loader.dataset.train else 'test'
    print('Checking accuracy on the %s set' % split)
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # 更换设备，如GPU
            y = y.to(device=device, dtype=torch.int64)
            scores = model_fn(x, params)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

### BareBones PyTorch: 训练循环
我们现在可以建立一个基本的训练循环来训练我们的网络。我们将使用无动量的随机梯度下降来训练模型。我们将使用`torch.function.cross_entropy`来计算损失； 你可以 [在这里阅读相关内容](http://pytorch.org/docs/stable/nn.html#cross-entropy).

训练循环将神经网络函数、初始化参数列表（在我们的示例中为`[w1, w2]`）和学习率作为输入。

In [13]:
def train_part2(model_fn, params, learning_rate):
    """
    在 CIFAR-10 上训练模型。
    
    输入:
    - model_fn:执行模型前向传播的 Python 函数。它应该具有签名 Score = model_fn(x, params) 
      其中 x 是图像数据的 PyTorch 张量，params 是 PyTorch 张量的列表模型权重，
      分数是形状 (N, C) 的 PyTorch 张量 x 中元素的分数。
    - params:赋予模型权重的 PyTorch 张量列表
    - learning_rate: Python 标量给出用于 SGD 的学习率
    
    返回: Nothing
    """
    for t, (x, y) in enumerate(loader_train):
        # 将数据移动到正确的设备（GPU 或 CPU）
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)

        # 前向传递：计算分数和损失
        scores = model_fn(x, params)
        loss = F.cross_entropy(scores, y)

        # 向后传递：PyTorch 找出计算图中哪些张量具有 require_grad=True 
        # 并使用反向传播来计算相对于这些张量的损失梯度，
        # 并将梯度存储在每个 Tensor 的 .grad 属性中。
        loss.backward()

        # 更新参数。 
        #我们不想通过参数更新进行反向传播，
        #因此我们将更新范围限制在 torch.no_grad() 上下文管理器下，以防止构建计算图。
        with torch.no_grad():
            for w in params:
                w -= learning_rate * w.grad

                # 运行反向传播后手动将梯度归零
                w.grad.zero_()

        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_part2(loader_val, model_fn, params)
            print()

### BareBones PyTorch: 训练两层网络
现在我们准备好运行训练循环了。我们需要为全连接权重`w1`和`w2`显式分配张量。 

CIFAR 的每个小批量有 64 个示例，因此张量形状为 `[64, 3, 32, 32]`。

展平后, `x` 的形状应为`[64, 3 * 32 * 32]`. 他将是`w1`第一个维度的大小。 
`w1`的第二个维度是隐藏层大小，这也是`w2`的第一个维度。 

最后，网络的输出是一个 10 维向量，表示 10 个类别的概率分布。

您不需要调整任何超参数，但在训练一个 epoch 后，您应该会看到准确率高于 40%。

In [14]:
hidden_layer_size = 4000
learning_rate = 1e-2

w1 = random_weight((3 * 32 * 32, hidden_layer_size))
w2 = random_weight((hidden_layer_size, 10))

train_part2(two_layer_fc, [w1, w2], learning_rate)

Iteration 0, loss = 3.1638
Checking accuracy on the val set
Got 141 / 1000 correct (14.10%)

Iteration 100, loss = 2.0802
Checking accuracy on the val set
Got 345 / 1000 correct (34.50%)

Iteration 200, loss = 2.5873
Checking accuracy on the val set
Got 364 / 1000 correct (36.40%)

Iteration 300, loss = 2.0787
Checking accuracy on the val set
Got 382 / 1000 correct (38.20%)

Iteration 400, loss = 2.0624
Checking accuracy on the val set
Got 419 / 1000 correct (41.90%)

Iteration 500, loss = 1.7178
Checking accuracy on the val set
Got 417 / 1000 correct (41.70%)

Iteration 600, loss = 2.2978
Checking accuracy on the val set
Got 427 / 1000 correct (42.70%)

Iteration 700, loss = 1.3888
Checking accuracy on the val set
Got 405 / 1000 correct (40.50%)



### BareBones PyTorch: 训练卷积网络

在下面，您应该使用上面定义的函数在 CIFAR 上训练三层卷积网络。网络应具有以下架构：

1. 具有 32 个 5x5 滤波器的卷积层（带偏置），零填充为 2
2. ReLU
3. 具有 16 个 3x3 滤波器的卷积层（带偏置），零填充为 1
4. ReLU
5. 全连接层（带有偏差）计算 10 个类别的分数

您应该使用上面定义的`random_weight`函数初始化权重矩阵，并且应该使用上面定义的`zero_weight`函数初始化偏差向量。

您不需要调整任何超参数，但如果一切正常，您应该在一个 epoch 后达到 42% 以上的准确率。

In [15]:
learning_rate = 3e-3

channel_1 = 32
channel_2 = 16

conv_w1 = None
conv_b1 = None
conv_w2 = None
conv_b2 = None
fc_w = None
fc_b = None

################################################################################
# TODO: 初始化三层ConvNet的参数。                                              #
################################################################################
#       *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****          #
################################################################################

conv_w1 = random_weight((channel_1, 3, 5, 5))
conv_b1 = zero_weight((channel_1,))
conv_w2 = random_weight((channel_2, 32, 3, 3))
conv_b2 = zero_weight((channel_2,))
fc_w = random_weight((channel_2*32*32, 10))
fc_b = zero_weight((10,))

################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
train_part2(three_layer_convnet, params, learning_rate)

Iteration 0, loss = 3.9098
Checking accuracy on the val set
Got 115 / 1000 correct (11.50%)

Iteration 100, loss = 1.7671
Checking accuracy on the val set
Got 360 / 1000 correct (36.00%)

Iteration 200, loss = 1.4617
Checking accuracy on the val set
Got 399 / 1000 correct (39.90%)

Iteration 300, loss = 1.9257
Checking accuracy on the val set
Got 447 / 1000 correct (44.70%)

Iteration 400, loss = 1.7655
Checking accuracy on the val set
Got 451 / 1000 correct (45.10%)

Iteration 500, loss = 1.4304
Checking accuracy on the val set
Got 486 / 1000 correct (48.60%)

Iteration 600, loss = 1.4907
Checking accuracy on the val set
Got 478 / 1000 correct (47.80%)

Iteration 700, loss = 1.5136
Checking accuracy on the val set
Got 486 / 1000 correct (48.60%)



# Part III. PyTorch Module API

Barebone PyTorch 要求我们手动跟踪所有参数张量。这对于具有几个张量的小型网络来说很好，但在较大的网络中跟踪数十或数百个张量将非常不方便且容易出错。

PyTorch 提供`nn.Module` API 供您定义任意网络架构，同时为您跟踪每个可学习的参数。 在Part II中, 我们自己实施了SGD(梯度下降). PyTorch  还提供了  `torch.optim` 包，它实现了所有常见的优化器， 例如 RMSProp, Adagrad 和 Adam. 它甚至支持近似二阶方法，如 L-BFGS！您可以参考[文档](http://pytorch.org/docs/master/optim.html)了解每个优化器的具体规格。

要使用模块 API，请按照以下步骤操作：

1. 子类`nn.Module`。为您的网络类指定一个直观的名称，例如`TwoLayerFC`。

2. 在构造函数`__init__()`中，将所需的所有层定义为类属性。像`nn.Linear`和`nn.Conv2d`这样的图层对象本身就是`nn.Module`子类，并且包含可学习的参数， 这样您就不必自己实例化原始张量。 `nn.Module` 将为您跟踪这些内部参数。 请参阅 [文档](http://pytorch.org/docs/master/nn.html)以了解有关数十个内置层的更多信息。 **警告**: 不要忘记先调用 `super().__init__()` ！

3. 在`forward()`方法中，定义网络的*连接性*。 您应该使用 __init__ 中定义的属性作为函数调用，以张量作为输入并输出“转换后的”张量。*不要* 在`forward()`中创建具有可学习参数的任何新层！所有这些都必须在“__init__”中预先声明。

定义模块子类后，您可以将其实例化为对象并调用它,就像Part II中的 NN 前向函数一样。

### Module API:双层网络
下面是一个 2 层全连接网络的具体示例：

In [16]:
class TwoLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        # 将图层对象分配给类属性
        self.fc1 = nn.Linear(input_size, hidden_size)
        # nn.init 包含便捷的初始化方法
        # http://pytorch.org/docs/master/nn.html#torch-nn-init 
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        nn.init.kaiming_normal_(self.fc2.weight)
    
    def forward(self, x):
        # forward 始终定义连接性
        x = flatten(x)
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores

def test_TwoLayerFC():
    input_size = 50
    x = torch.zeros((64, input_size), dtype=dtype)  # 小批量大小 64, 特征维度 50
    model = TwoLayerFC(input_size, 42, 10)
    scores = model(x)
    print(scores.size())  # 你应该看到 [64, 10]
test_TwoLayerFC()

torch.Size([64, 10])


### Module API: 三层卷积网络
现在轮到你实现一个 3 层 ConvNet，然后是一个全连接层。网络架构应该与Part II相似:

1. 具有“channel_1”5x5 滤波器的卷积层，零填充为 2
2. ReLU
3. 具有“channel_2” 3x3 滤波器的卷积层，零填充为 1
4. ReLU
5. 到“num_classes”类的全连接层

您应该使用 Kaiming 法线初始化方法来初始化模型的权重矩阵。

**提示**: http://pytorch.org/docs/stable/nn.html#conv2d

实现三层 ConvNet 后, `test_ThreeLayerConvNet` 函数将运行您的实现; 它应该打印`(64, 10)`作为输出分数的形状。

In [17]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        ########################################################################
        # TODO: 使用上面定义的架构设置三层 ConvNet 所需的层                      #
        ########################################################################
        #*****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****         #
        ########################################################################
        
        self.conv1 = nn.Conv2d(in_channel, channel_1, kernel_size=5, padding=2, bias=True)
        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.constant_(self.conv1.bias, 0)
        
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1, bias=True)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.constant_(self.conv2.bias, 0)
        
        self.fc = nn.Linear(channel_2*32*32, num_classes)
        nn.init.kaiming_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: 实现 3 层 ConvNet 的前向功能。您应该使用在 __init__ 中定义的层   #
        # 并在forward() 中指定这些层的连接                                      #
        ########################################################################
        #*****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****         #
        ########################################################################
        
        relu1 = F.relu(self.conv1(x))
        relu2 = F.relu(self.conv2(relu1))
        scores = self.fc(flatten(relu2))
        
        ########################################################################
        #                          END OF YOUR CODE                            #
        ########################################################################
        return scores


def test_ThreeLayerConvNet():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # 小批量大小 64, 图片大小 [3, 32, 32]
    model = ThreeLayerConvNet(in_channel=3, channel_1=12, channel_2=8, num_classes=10)
    scores = model(x)
    print(scores.size())  # 你应该看到 [64, 10]
test_ThreeLayerConvNet()

torch.Size([64, 10])


### Module API: 检查准确性
给定验证或测试集，我们可以检查神经网络的分类准确性。

这个版本与第二部分的版本略有不同。您不再手动传递参数。

In [18]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # 将模型设置为评估模式
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # 转换设备，例如GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

### Module API: 训练循环
我们还使用了稍微不同的训练循环, 我们不是自己更新权重值，而是使用`torch.optim`包中的 Optimizer 对象，该对象抽象了优化算法的概念，并提供了大多数常用于优化神经网络的算法的实现。

In [19]:
def train_part34(model, optimizer, epochs=1):
    """
    使用 PyTorch 模块 API 在 CIFAR-10 上训练模型。
    
    输入:
    - model: 提供模型训练的 PyTorch Model。
    - optimizer: 我们将使用一个 Optimizer 对象来训练模型。
    - epochs: (可选) 一个 Python 整数，给定要训练的次数
    
    返回: Nothing, 但在训练期间打印模型精度。
    """
    model = model.to(device=device)  # 将模型参数移至CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # 将模型置于训练模式
            x = x.to(device=device, dtype=dtype)  # 转换设备，例如GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # 将优化器将更新的变量的所有梯度归零。
            optimizer.zero_grad()

            # 这是向后传递：计算模型每个参数的损失梯度。
            loss.backward()

            # 实际上使用向后传递计算的梯度来更新模型的参数。
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

### Module API: 训练双层网络
现在我们准备好运行训练循环了。与Part II相反，我们不再显式分配参数张量。

只需将输入大小、隐藏层大小和类数（即输出大小）传递给`TwoLayerFC`的构造函数即可。

您还需要定义一个优化器来跟踪`TwoLayerFC`内的所有可学习参数。

您不需要调整任何超参数，但在训练一个 epoch 后，您应该会看到模型准确率高于 40%。

In [20]:
hidden_layer_size = 4000
learning_rate = 1e-2
model = TwoLayerFC(3 * 32 * 32, hidden_layer_size, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_part34(model, optimizer)

Iteration 0, loss = 3.2762
Checking accuracy on validation set
Got 119 / 1000 correct (11.90)

Iteration 100, loss = 2.6811
Checking accuracy on validation set
Got 332 / 1000 correct (33.20)

Iteration 200, loss = 1.9052
Checking accuracy on validation set
Got 381 / 1000 correct (38.10)

Iteration 300, loss = 1.9003
Checking accuracy on validation set
Got 386 / 1000 correct (38.60)

Iteration 400, loss = 1.8123
Checking accuracy on validation set
Got 422 / 1000 correct (42.20)

Iteration 500, loss = 2.0043
Checking accuracy on validation set
Got 360 / 1000 correct (36.00)

Iteration 600, loss = 1.8639
Checking accuracy on validation set
Got 442 / 1000 correct (44.20)

Iteration 700, loss = 1.9156
Checking accuracy on validation set
Got 466 / 1000 correct (46.60)



### Module API: 训练三层卷积网络
您现在应该使用模块 API 在 CIFAR 上训练三层 ConvNet。这看起来应该与训练两层网络非常相似！你不需要调整任何超参数，但训练一个 epoch 后应该达到 45% 以上。

您应该使用没有动量的随机梯度下降来训练模型。

In [21]:
learning_rate = 3e-3
channel_1 = 32
channel_2 = 16

model = None
optimizer = None
################################################################################
# TODO: 实例化您的 ThreeLayerConvNet 模型和相应的优化器                          #
################################################################################
#*****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****                 #
################################################################################

model = ThreeLayerConvNet(3, channel_1, channel_2, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

################################################################################
#                             END OF YOUR CODE                                 #
################################################################################

train_part34(model, optimizer)

Iteration 0, loss = 3.8929
Checking accuracy on validation set
Got 144 / 1000 correct (14.40)

Iteration 100, loss = 1.9717
Checking accuracy on validation set
Got 348 / 1000 correct (34.80)

Iteration 200, loss = 1.8484
Checking accuracy on validation set
Got 394 / 1000 correct (39.40)

Iteration 300, loss = 1.6329
Checking accuracy on validation set
Got 420 / 1000 correct (42.00)

Iteration 400, loss = 1.8622
Checking accuracy on validation set
Got 466 / 1000 correct (46.60)

Iteration 500, loss = 1.7899
Checking accuracy on validation set
Got 457 / 1000 correct (45.70)

Iteration 600, loss = 1.5297
Checking accuracy on validation set
Got 476 / 1000 correct (47.60)

Iteration 700, loss = 1.3070
Checking accuracy on validation set
Got 497 / 1000 correct (49.70)



# Part IV. PyTorch Sequential API

Part III 引入了 PyTorch Module API, 它允许您定义任意可学习层及其连接性。

对于简单模型，例如一堆前馈层, 你还需要完成3个步骤: 子类 `nn.Module`, 在`__init__`中将层分配给类属性，并在`forward()`中逐层调用每一层。 

有没有更便捷的方法呢？

幸运的是, PyTorch 提供了一个名为 `nn.Sequential`的容器模块，, 它将上述步骤合并为一个. 它不像`nn.Module`那么灵活，因为您无法指定比前馈堆栈更复杂的拓扑，但它对于许多用例来说已经足够了。

### Sequential API: 双层网络
让我们看看如何使用`nn.Sequential`重写我们的两层全连接网络示例，并使用上面定义的训练循环来训练它。

同样，您不需要在这里调整任何超参数，但经过一个 epoch 的训练后，您应该达到 40% 以上的准确率。

In [22]:
# 我们需要将`flatten`函数包装在模块中，以便将其堆叠在 nn.Sequential 中
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

hidden_layer_size = 4000
learning_rate = 1e-2

model = nn.Sequential(
    Flatten(),
    nn.Linear(3 * 32 * 32, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 10),
)

# 您可以在 optim.SGD 中使用 Nesterov 动量
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model, optimizer)

Iteration 0, loss = 2.3243
Checking accuracy on validation set
Got 174 / 1000 correct (17.40)

Iteration 100, loss = 1.8203
Checking accuracy on validation set
Got 390 / 1000 correct (39.00)

Iteration 200, loss = 1.7088
Checking accuracy on validation set
Got 388 / 1000 correct (38.80)

Iteration 300, loss = 1.6694
Checking accuracy on validation set
Got 406 / 1000 correct (40.60)

Iteration 400, loss = 1.8396
Checking accuracy on validation set
Got 426 / 1000 correct (42.60)

Iteration 500, loss = 1.9302
Checking accuracy on validation set
Got 422 / 1000 correct (42.20)

Iteration 600, loss = 1.8641
Checking accuracy on validation set
Got 423 / 1000 correct (42.30)

Iteration 700, loss = 1.5718
Checking accuracy on validation set
Got 446 / 1000 correct (44.60)



### Sequential API: 三层卷积网络
在这里，您应该使用`nn.Sequential`来定义和训练一个三层 ConvNet，其架构与我们在Part III中使用的架构相同：

1. 具有 32 个 5x5 滤波器的卷积层（带偏置），零填充为 2
2. ReLU
3. 具有 16 个 3x3 滤波器的卷积层（带偏置），零填充为 1
4. ReLU
5. 全连接层（带有偏差）计算 10 个类别的分数

您应该使用上面定义的`random_weight`函数初始化权重矩阵，并且应该使用上面定义的`zero_weight`函数初始化偏差向量。

您应该使用 Nesterov 动量 0.9 的随机梯度下降来优化您的模型。

同样，您不需要调整任何超参数，但经过一轮训练后您应该会看到准确率高于 55%。

In [23]:
def kaiming_normal(shape):
    """
    为权重创建随机张量；设置 require_grad=True 意味着我们要在向后传递过程中计算这些张量的梯度。
    我们使用 Kaiming 初始化: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[1]  # 与 `random_weight()` 不同，pytorch 中 nn.Linear 的权重形状为：[out_feature, in_feature]
    else:
        fan_in = np.prod(shape[1:]) # 转换权重 [out_channel, in_channel, kH, kW]
    # randn 是标准正态分布生成器。
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def xavier_normal(shape):
    """
    为权重创建随机张量；设置 require_grad=True 意味着我们要在向后传递过程中计算这些张量的梯度。
    我们使用 Xavier 初始化: sqrt(2 / (fan_in + fan_out))
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[1]
        fan_out = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # 转换权重 [out_channel, in_channel, kH, kW]
        fan_out = shape[0] * shape[2] * shape[3]
    # randn 是标准正态分布生成器。
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / (fan_in + fan_out))
    w.requires_grad = True
    return w

In [24]:
channel_1 = 32
channel_2 = 16
learning_rate = 1e-2

model = None
optimizer = None

################################################################################
# TODO:使用 Sequential API 重写第三部分中带有偏差的 3 层 ConvNet。               #                                                        #
################################################################################
#*****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****                 #
################################################################################

model = nn.Sequential(
    nn.Conv2d(3, channel_1, kernel_size=5, padding=2),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2*32*32, 10),
)

optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

# 权重初始化
# 参考: http://pytorch.org/docs/stable/nn.html#torch.nn.Module.apply
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
#         m.weight.data = random_weight(m.weight.size())
#         m.weight.data = kaiming_normal(m.weight.size())
        m.weight.data = xavier_normal(m.weight.size())
        m.bias.data = zero_weight(m.bias.size())

model.apply(init_weights)

################################################################################
#                            END OF YOUR CODE                                  #                           
################################################################################

train_part34(model, optimizer)

Iteration 0, loss = 2.4132
Checking accuracy on validation set
Got 158 / 1000 correct (15.80)

Iteration 100, loss = 1.6887
Checking accuracy on validation set
Got 429 / 1000 correct (42.90)

Iteration 200, loss = 1.3527
Checking accuracy on validation set
Got 487 / 1000 correct (48.70)

Iteration 300, loss = 1.4333
Checking accuracy on validation set
Got 489 / 1000 correct (48.90)

Iteration 400, loss = 1.4937
Checking accuracy on validation set
Got 524 / 1000 correct (52.40)

Iteration 500, loss = 1.2148
Checking accuracy on validation set
Got 528 / 1000 correct (52.80)

Iteration 600, loss = 1.2533
Checking accuracy on validation set
Got 545 / 1000 correct (54.50)

Iteration 700, loss = 1.1077
Checking accuracy on validation set
Got 572 / 1000 correct (57.20)



# Part V. CIFAR-10 开放式挑战

在本节中，您可以在 CIFAR-10 上试验您想要的任何 ConvNet 架构。

现在，您的工作是试验架构、超参数、损失函数和优化器，以训练一个模型，使其在 10 个 epoch 内的 CIFAR-10 **验证集** 上实现 **至少 70%** 的准确度。您可以使用上面的check_accuracy 和 train 函数。您可以使用`nn.Module`或`nn.Sequential`API。

描述一下你在本笔记本末尾做了什么。

以下是每个组件的官方 API 文档。需要注意的是：我们在`空间批规范`类中所说的在 PyTorch 中称为`BatchNorm2D`。

* torch.nn 包中的层: http://pytorch.org/docs/stable/nn.html
* 激活函数: http://pytorch.org/docs/stable/nn.html#non-linear-activations
* 损失函数: http://pytorch.org/docs/stable/nn.html#loss-functions
* Optimizers优化器: http://pytorch.org/docs/stable/optim.html


### 你可以尝试的事情:
- **过滤器尺寸**: 上面我们使用了5x5；较小的过滤器是否会更有效？
- **过滤器数量**: 上面我们使用了 32 个过滤器。是越多越好还是越少越好？
- **池化与跨步卷积**: 您使用最大池化还是仅使用跨步卷积？
- **批量归一化**: 尝试在卷积层之后添加空间批量归一化，并在仿射层之后添加普通批量归一化。您的网络训练得更快吗？
- **网络架构**: 上面的网络有两层可训练参数。你能用深度网络做得更好吗？值得尝试的良好架构包括：
    - [卷积-Relu-池化]xN -> [仿射]xM -> [softmax or SVM]
    - [卷积-Relu-卷积-Relu-池化]xN -> [仿射]xM -> [softmax 或 SVM]
    - [批量归一化-Relu-卷积]xN -> [仿射]xM -> [softmax 或 SVM]
- **全局平均池**: 不要展平然后拥有多个仿射层，而是执行卷积直到图像变小（7x7 左右），然后执行平均池化操作以获得 1x1 图像图片（1, 1 , Filter#），然后将其重新整形为一个 (Filter#) 向量。 这在谷歌的初始网络[Google's Inception Network](https://arxiv.org/abs/1512.00567)中使用 (其架构见表 1).
- **正则化**: 添加 l2 权重正则化，或者可能使用 Dropout。
### 关于训练的小提示
对于您尝试的每个网络架构，您应该调整学习率和其他超参数。执行此操作时，需要记住以下重要事项：

- 如果参数运行良好，您应该会在几百次迭代内看到改进
- 请记住超参数调整的从粗到细的方法：首先测试大量的超参数，进行几次训练迭代，以找到有效的参数组合。
- 一旦找到一些似乎有效的参数集，就可以围绕这些参数进行更精细的搜索。您可能需要训练更多次数。
- 您应该使用验证集进行超参数搜索，并保存测试集，以便根据验证集选择的最佳参数评估您的架构。

### 改进、提高！
如果您喜欢探索，您可以实施许多其他功能来尝试提高您的表现。 **不需要**实现其中任何一个，但如果您有时间，请不要错过其中的乐趣！

- 替代优化器：你可以尝试 Adam、Adagrad、RMSprop 等。
- 替代激活函数，例如leaky ReLU、参数化ReLU、ELU 或MaxOut。
- Model整合
- 数据增强
- 新架构
  - [ResNets](https://arxiv.org/abs/1512.03385) 其中前一层的输入被添加到输出中。
  - [DenseNets](https://arxiv.org/abs/1608.06993) 其中前一层的输入连接在一起。
  - [This blog has an in-depth overview](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

### 享受愉快、快乐的训练! 

In [25]:
################################################################################
# TODO:                                                                        #         
#  使用任何架构、优化器和超参数进行实验。                                        #
#  在 10 个周期内的“验证集”上实现至少 70% 的准确率。                             #
#  请注意，您可以使用 check_accuracy 函数对测试集或验证集进行评估                 #
#  方法是将 loader_test 或 loader_val 作为第二个参数传递给 check_accuracy。      #
#  在完成架构和超参数调整之前，不应触摸测试集                                    # 
#  并且仅在最后运行测试集一次以报告最终值。                                      #
################################################################################
#*****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****                 #
################################################################################
model = None
optimizer = None

# 4层卷积网络
# (卷积 -> 批量归一化 -> Relu -> max池化) * 3 -> fc
layer1 = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=5, padding=2),
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.MaxPool2d(2)
)

layer2 = nn.Sequential(
    nn.Conv2d(16, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2)
)

layer3 = nn.Sequential(
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2)
)

fc = nn.Linear(64*4*4, 10)

model = nn.Sequential(
    layer1,
    layer2,
    layer3,
    Flatten(),
    fc
)

learning_rate = 1e-3

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 每个时期打印训练状态：将 print_every 设置为一个大数字
print_every = 10000

################################################################################
#                                END OF YOUR CODE                              #                       
################################################################################

# 您应该获得至少 70% 的准确率
train_part34(model, optimizer, epochs=10)

Iteration 0, loss = 2.4207
Checking accuracy on validation set
Got 105 / 1000 correct (10.50)

Iteration 0, loss = 0.8765
Checking accuracy on validation set
Got 628 / 1000 correct (62.80)

Iteration 0, loss = 0.7449
Checking accuracy on validation set
Got 693 / 1000 correct (69.30)

Iteration 0, loss = 0.6328
Checking accuracy on validation set
Got 702 / 1000 correct (70.20)

Iteration 0, loss = 0.6688
Checking accuracy on validation set
Got 718 / 1000 correct (71.80)

Iteration 0, loss = 0.5392
Checking accuracy on validation set
Got 754 / 1000 correct (75.40)

Iteration 0, loss = 0.6220
Checking accuracy on validation set
Got 723 / 1000 correct (72.30)

Iteration 0, loss = 0.4938
Checking accuracy on validation set
Got 760 / 1000 correct (76.00)

Iteration 0, loss = 0.4196
Checking accuracy on validation set
Got 770 / 1000 correct (77.00)

Iteration 0, loss = 0.6582
Checking accuracy on validation set
Got 748 / 1000 correct (74.80)



## 描述一下你做了什么

在下面的单元格中，您应该解释您所做的事情、您实现的任何附加功能和/或您在训练和评估网络的过程中制作的任何图表。

TODO: 描述一下你做了什么

## 测试集——仅运行一次

现在我们已经得到了满意的结果，我们在测试集上测试我们的最终模型（您应该将其存储在 best_model 中）。考虑一下这与您的验证集准确性相比如何。

In [26]:
best_model = model
check_accuracy_part34(loader_test, best_model)

Checking accuracy on test set
Got 7331 / 10000 correct (73.31)
